In [ ]:
function field(t,x,z,h)
    hT=h; # hight of transmitter (source)
    ρF=1000.
    cF=1480
    #K=rhoF*cF^2
    ρS=2530.
    cP=5960.
    cS=3460.
    #μ=ρS*cS^2
    #λ=ρS*cP^2-2*μ
    #Incident wave
    Tfi=sqrt(x^2+(hT-z)^2)/cF
    if t>Tfi
        Gi=1/(2π*sqrt((t+0im)^2-Tfi^2))
    else
        Gi=0
    end

    #reflected wave
    Tff=sqrt(x^2+(hT+z)^2)/cF
    Tfp=(x/cP)+sqrt(1/cF^2-1/cP^2)*(hT+z)
    # I don't want to use p as in papers not to mix with pressure p≡r
    if t>Tff
        r=(x*t+1im*(hT+z)*sqrt((t+0im)^2-Tff^2))/(x^2+(hT+z)^2)
    elseif Tfp<t<Tff
        r=(x*t-(hT+z)*sqrt(Tff^2-(t+0im)^2))/(x^2+(hT+z)^2)
    else
        r=0
    end

    if t>Tfp
        γF=sqrt(1/cF^2-r^2)
        γS=sqrt(1/cS^2-r^2)
        γP=sqrt(1/cP^2-r^2)

        ΔR=(r^2-1/2cS^2)^2+r^2 *γP*γS # Rayleigh-wave denomenator
        ΔS=(ρF*γP/(4*ρS*γF*cS^4))+ΔR # Schol-wave denomenator

        Rf=((-ρF*γP/(4*ρS*γF*cS^4))+ΔR)/ΔS
    end


    #Gr fluid
    if t>Tff
        Gr=real(Rf)/(2π*sqrt((t+0im)^2-Tff^2))
    elseif Tfp<t<Tff
        Gr=imag(Rf)/(2π*sqrt(Tff^2-(t+0im)^2))
    else
        Gr=0
    end
    return Gi, Gr
end
#########################################
function getP(h)
    ρF=1000.
    f0=1.5e6
    t0=1.e-6
    x=linspace(0,1e-2,201)
    z=linspace(0,1e-2,201)
    τ=linspace(0,17*9*30t0,17*9*6000+1)
    fr= -sqrt(2)*π*f0*(τ-t0).*exp(0.5-(π*f0*(τ-t0)).^2)
    Δt=τ[3]-τ[2]
    #τ=[0:Δt:3*15t0]
    p=zeros(length(z),length(x),601 )#length(τ))
    #Gi=zeros(length(z),length(x),length(τ))
    #Gr=zeros(length(z),length(x),length(τ))
    gi=zeros(length(τ))
    gr=zeros(length(τ))
    #tic()
    for j=1:length(x)
        for k=1:length(z)
            for i=1:length(τ)
                gi[i], gr[i]= field(τ[i], x[j], z[k], h);
            end
            gi=sign(gi).*min(abs(gi), 1e6)
            gr=sign(gr).*min(abs(gr), 1e6)
            Pi=ρF*Δt*conv(real(gi),fr)
            Pr=ρF*Δt*conv(real(gr),fr)
            P=Pi+Pr
            P=P[1:10:6000+1]
            p[k,j,:]=P
                #Gr[k, j,:]=gr[1:length(τ)]
                #Gi[k, j,:]=gi[1:length(τ)]
        end
    end
    return p
end
####################get for one pont#########
function getPoint(h,x,z,th)
    ρF=1000.
    f0=1.5e6
    t0=1.e-6
    τ=linspace(0,1*30t0,1*5*6000+1)
    fr= -sqrt(2)*π*f0*(τ-t0).*exp(0.5-(π*f0*(τ-t0)).^2)
    Δt=τ[3]-τ[2]
    #τ=[0:Δt:3*15t0]
    #p=zeros(1201 )#length(τ))
    #Gi=zeros(length(z),length(x),length(τ))
    #Gr=zeros(length(z),length(x),length(τ))
    gi=zeros(length(τ))
    gr=zeros(length(τ))
    #tic()

    for i=1:length(τ)
        gi[i], gr[i]= field(τ[i], x , z, h);
    end
    gi=sign(gi).*min(abs(gi), th) # th=threshold
    gr=sign(gr).*min(abs(gr), th)
    Pi=ρF*Δt*conv(real(gi),fr)
    Pr=ρF*Δt*conv(real(gr),fr)
    P=Pi+Pr
    p=vec(P[1:5*5:5*6000+1])
    return p
end
